In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Task Details
Following are the inspirations to work in this dataset :
1.	How many states have fully vaccinated?
2.	What is the number of positive cases rising in India?
3.	Which states are more affected by the virus?
4.	Analyze to show insights regarding coronavirus

In [ ]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import os
import glob
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
############################# Read Covid_19_india data into the datatframe Covid19India #######################################
print("Read Covid_19_india data into the dataframe Covid19India\n")
Covid19India = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')

In [ ]:
print("How does the data look like\n", Covid19India.head())

In [ ]:
print("The columns in the data\n", list(Covid19India.columns))

In [ ]:
print("We can drop 2 columns ['ConfirmedIndianNational', 'ConfirmedForeignNational']\n")
Covid19India = Covid19India.drop(['ConfirmedIndianNational', 'ConfirmedForeignNational'], axis = 1)

In [ ]:
print("## Verify to see whether the 2 columns are dropped\n",Covid19India.head())

In [ ]:
print("Some Infomation \n", Covid19India.info())

There are 14,690 rows of data of Cured, Deaths and Confirmed numbers distributed statewise from 30th Jan 2020 till 03rd May 2021. We apply pd.to_datetime to 'Date'

In [ ]:
Covid19India['Date'] = pd.to_datetime(Covid19India['Date'], format = '%Y-%m-%d')
Covid19India.columns = ['Sno', 'Date', 'Time', 'State','Cured','Deaths','Confirmed']
print("No of States: ",Covid19India.State.nunique())
print("The states are: \n", Covid19India.State.unique())

There are 2 instances of 'State' that we shall drop, 'Cases being reassigned to states' and 'Unassigned'

In [ ]:
dropIndex = Covid19India.loc[Covid19India['State'] == 'Cases being reassigned to states'].index
Covid19India.drop(dropIndex, inplace = True)
dropIndex = Covid19India.loc[Covid19India['State'] == 'Unassigned'].index
Covid19India.drop(dropIndex, inplace = True)
print("After dropping 'Cases being reassigned to states' and 'Unassigned,the dataframe has :\n",Covid19India.shape)

In [ ]:
########################### Sorting Covid19India in descending order of 'Date' and 'Confirmed' ###############################
######################################## into Covid19ToDate ##########################################################

Covid19ToDate = Covid19India.sort_values(['Date', 'Confirmed'],ascending = False)
Covid19ToDate.drop('Sno', axis =1)

In [ ]:
################## Plot the curve of Confirmed, Cured and Death cases in India from Feb 2020 till date #########################

print("Plotting the curve of Confirmed, Cured and Death Cases in India from Feb 2020 till date\n\n")

Covid19ByDay = Covid19India.groupby('Date')['Cured', 'Deaths', 'Confirmed'].sum()
Covid19ByDay['CuredPerDay'] = Covid19ByDay.Cured - Covid19ByDay.Cured.shift(1)
Covid19ByDay['DeathsPerDay'] = Covid19ByDay.Deaths - Covid19ByDay.Deaths.shift(1)
Covid19ByDay['ConfirmedPerDay'] = Covid19ByDay.Confirmed - Covid19ByDay.Confirmed.shift(1)

## Calculate Rolling Mean of 7 days
Covid19ByDay['CuredPerDay7MM'] = Covid19ByDay['CuredPerDay'].rolling(window = 7).mean()
Covid19ByDay['DeathsPerDay7MM'] = Covid19ByDay['DeathsPerDay'].rolling(window = 7).mean()
Covid19ByDay['ConfirmedPerDay7MM'] = Covid19ByDay['ConfirmedPerDay'].rolling(window = 7).mean()

Covid19ByDay = Covid19ByDay[(Covid19ByDay.index >= '2020-02-01')]


fig,ax = plt.subplots(ncols=1,nrows=3,dpi=100,figsize=(20,18))


ax[0].plot(Covid19ByDay.Confirmed, color = 'dodgerblue')
ax[0].plot(Covid19ByDay.Cured, color = 'green')
ax[0].plot(Covid19ByDay.Deaths, color = 'red')
ax[0].legend(['Confirmed','Cured', 'Deaths'], prop={'size': 20})
ax[0].set_ylabel("Cumulative graph \n In Crores", fontsize=16)
ax[0].tick_params(axis='both', labelsize=16)

ax[0].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax[0].xaxis.set_major_formatter(DateFormatter('%b%y'))
for label in ax[0].xaxis.get_ticklabels():
            label.set_rotation(45)

Xstart, Xend = ax[0].get_xlim()
ax[0].hlines(y=[0.5e+7, 1.0e+7, 1.5e+7], xmin=Xstart, xmax=Xend,  color='gray', linewidth = 0.7)


ax[1].plot(Covid19ByDay.ConfirmedPerDay, color = 'dodgerblue')
ax[1].plot(Covid19ByDay.CuredPerDay, color = 'green')
ax[1].plot(Covid19ByDay.DeathsPerDay, color = 'red')
ax[1].legend(['Confirmed','Cured', 'Deaths'], prop={'size': 20})
ax[1].set_ylabel("Daywise pattern", fontsize =16)
ax[1].tick_params(axis='both', labelsize=16)

ax[1].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax[1].xaxis.set_major_formatter(DateFormatter('%b%y'))
for label in ax[1].xaxis.get_ticklabels():
            label.set_rotation(45)
        
Xstart, Xend = ax[1].get_xlim()
ax[1].hlines(y=[100000, 200000, 300000], xmin=Xstart, xmax=Xend,  color='gray', linewidth = 0.7)




ax[2].plot(Covid19ByDay.ConfirmedPerDay7MM, color = 'dodgerblue')
ax[2].plot(Covid19ByDay.CuredPerDay7MM, color = 'green')
ax[2].plot(Covid19ByDay.DeathsPerDay7MM, color = 'red')
ax[2].legend(['Confirmed','Cured', 'Deaths'], prop={'size': 20})
ax[2].set_ylabel('7 Day Rolling Average', fontsize =16)
ax[2].tick_params(axis='both', labelsize=16)

ax[2].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax[2].xaxis.set_major_formatter(DateFormatter('%b%y'))
for label in ax[2].xaxis.get_ticklabels():
            label.set_rotation(45)
        
Xstart, Xend = ax[2].get_xlim()
ax[2].hlines(y=[100000, 200000, 300000], xmin=Xstart, xmax=Xend,  color='gray', linewidth = 0.7)


plt.show()

The cummulative graph of India shows a steady increase in cases from June 2020 to Oct 2020 and then a sudden spike in cases in the month of Oct-Nov 2020. The graph was steady for a while till early April 2021 when there was a very sudden increase in Covid19 cases.
This is corroborated from the next "Daywise Pattern" graph.
The last "7 Day Rolling Average" graph gives a much clearer picture. There is an enormous increase in Covid19 cases in India from early April 2021. The graph is started to taper towards the end signifying that the worst might be behind us.

In [ ]:
### Plot the 5 Worst Affected States 
print("What are the worst 5 affected states of India?\n")
LatestDate = Covid19ToDate.reset_index().Date[0]
Covid19ToDate = Covid19ToDate[Covid19ToDate['Date'] == LatestDate].sort_values(by='Confirmed', ascending=False)

fig,ax = plt.subplots(ncols=3,nrows=1,dpi=100,figsize=(18,8))
CovidWorstStates = Covid19ToDate.head(5)

sns.barplot(data = CovidWorstStates, x='State', y='Confirmed', ax=ax[0])
ax[0].set_yticklabels([5, 10, 15, 20, 25, 30, 35, 40, 45])
ax[0].set_ylabel('Confirmed (in lakhs)', fontsize =15)
ax[0].set_xticklabels(CovidWorstStates.State, rotation=90, fontsize = 14)
ax[0].set_xlabel(" ")

sns.barplot(data = CovidWorstStates, x='State', y='Cured', ax=ax[1])
ax[1].set_yticklabels([5, 10, 15, 20, 25, 30, 35, 40])
ax[1].set_ylabel('Cured (in lakhs)', fontsize =15)
ax[1].set_xticklabels(CovidWorstStates.State, rotation=90, fontsize = 14)
ax[1].set_xlabel(" ")


sns.barplot(data = CovidWorstStates, x='State', y='Deaths', ax=ax[2])
ax[2].set_ylabel('Deaths', fontsize =15)
ax[2].set_xticklabels(CovidWorstStates.State, rotation=90, fontsize = 14)
ax[2].set_xlabel(" ")

##plt.suptitle("Worst Indian States Affected", fontsize=20, fontweight='bold')
plt.show()

The worst affected states are : Maharashtra, Karnataka, Kerala, Uttar Pradesh and Tamil Nadu in that order

In [ ]:
### Lets look at the data of top 5 states more closely - Maharashtra, Kerala, Karnataka, UP, Tamil Nadu.
# Gathering data of 5 states

print("How is the situation in 5 worst affected states?\n")

CovidStates = list(CovidWorstStates.State.head(5))
col = ['teal', 'green', 'tomato', 'purple', 'mediumseagreen']

fig, ax = plt.subplots(nrows=5, ncols=1, dpi=100, figsize=(20,18))

for i in range(len(CovidStates)):
    CovidData = Covid19India[Covid19India['State'] == CovidStates[i]]
    CovidData['CuredPerDay'] = CovidData.Cured - CovidData.Cured.shift(1)
    CovidData['DeathsPerDay'] = CovidData.Deaths - CovidData.Deaths.shift(1)
    CovidData['ConfirmedPerDay'] = CovidData.Confirmed - CovidData.Confirmed.shift(1)
    CovidData = CovidData.fillna(0)
            
    sns.lineplot(x=CovidData.Date, y=CovidData.CuredPerDay.rolling(window=7).mean(), label = 'Cured',color='dodgerblue', ax=ax[i])
    sns.lineplot(x=CovidData.Date, y=CovidData.DeathsPerDay.rolling(window=7).mean(), label = 'Deaths', color='red', ax=ax[i])
    sns.lineplot(x=CovidData.Date, y=CovidData.ConfirmedPerDay.rolling(window=7).mean(), label = 'Confirmed',color='green', ax=ax[i])
    ax[i].set_ylabel(CovidStates[i], fontsize=20)
    ax[i].set_xlabel(" ")
    ax[i].tick_params(axis='both', labelsize=14)
    ax[i].legend(['Cured', 'Deaths','Confirmed'], prop={'size': 14})
    ax[i].xaxis.set_major_locator(mdates.MonthLocator(interval=2))
    ax[i].xaxis.set_major_formatter(DateFormatter('%b%y'))
    Xstart, Xend = ax[i].get_xlim()
    Ystart, Yend = ax[i].get_ylim()
    Ystep = round((Yend-Ystart)/3, 2)
    ax[i].hlines(y=[Ystep, 2*Ystep, 3*Ystep], xmin=Xstart, xmax=Xend,  color='gray', linewidth = 0.7)
    
    
#plt.sca(ax[0])
#plt.title('Cases Affected ', fontsize=24)

plt.show()


From the above graph we can say that the worst is over for the states Maharashtra and Tamil Nadu as of today. 
The state of Kerala is reaching its peak.
But the southern states of Tamil Nadu and Karnataka are still having their bad days.

In [ ]:
######################################################################################################
############################## How bad is the situation of 7 worst States? ##########################
######################################################################################################

print("Comparing the worst affected states with rest of India\n")

NumsConfirmed = list(Covid19ToDate['Confirmed'][:5].values)
NumsDeaths = list(Covid19ToDate.sort_values(by='Deaths', ascending = False)['Deaths'][:5])
labels = list(Covid19ToDate['State'][:5].values)

NumsConfirmed.append(Covid19ToDate['Confirmed'][5:].sum())
NumsDeaths.append(Covid19ToDate.sort_values(by='Deaths', ascending = False)['Deaths'][5:].sum())
labels.append('Rest of India')

fig, ax = plt.subplots(ncols=2, nrows=1, figsize = (16,16))

ax[0].pie(NumsConfirmed, labels=labels, autopct = '%1.1f%%', explode=[0.05]*len(labels))
ax[0].set_title('% share of Confirmed Cases', fontsize = 16)

ax[1].pie(NumsDeaths, labels=labels, autopct = '%1.1f%%', explode=[0.05]*len(labels))
ax[1].set_title('% share of Deaths', fontsize = 16)

plt.show()

In [ ]:
####################################################################################################
######################################### All About Cured  ###########################################
######################################################################################################
print("There are some good news too\n")

NumsCured = list(Covid19ToDate.sort_values(by='Cured', ascending = False)['Cured'][:5])
NumsCured.append(Covid19ToDate.sort_values(by='Cured', ascending = False)['Cured'][5:].sum())

labels = list(Covid19ToDate['State'][:5].values)
labels.append('Rest of India')

fig, ax = plt.subplots(ncols=1, nrows=1, figsize = (8,8))

ax.pie(NumsDeaths, labels=labels, autopct = '%1.1f%%', explode=[0.05]*len(labels))
ax.set_title('% share of Cured', fontsize = 16)

plt.show()

In [ ]:
#############################################################################################################################
############## We compare the mean of Confirmed cases of last 1 month(30 days) with the Confirmed case of latestDate ##########
###############################################################################################################################

print("For 5 Worst Affected States\n")
print("Comparing the Mean of Confirmed Cases of last 1 month(30 days) with Confirmed Cases of latest date")
CovidStates = list(CovidWorstStates.State.head(5))
col = ['teal', 'green', 'tomato', 'purple', 'mediumseagreen']


fig, ax = plt.subplots(nrows=1, ncols=1, dpi=100, figsize=(6,4))

for i in range(len(CovidStates)):
    CovidStateData = Covid19India[(Covid19India['State']==CovidStates[i])].reset_index().sort_values(by='index',ascending=False)
    CovidStateData['ConfirmedPerDay'] = CovidStateData.Confirmed.shift(1) - CovidStateData.Confirmed
    StateMean = round(CovidStateData['ConfirmedPerDay'].fillna(0).rolling(window=30).mean().values[29])
    StateLatest = CovidStateData.reset_index().ConfirmedPerDay[1]
    sns.scatterplot(y = i, x= [StateMean], color = 'mediumseagreen', s=50, edgecolor='black',ax=ax)
    sns.scatterplot(y= i, x= [StateLatest], color = 'coral', edgecolor='black', s = 50, ax=ax)

ax.set_yticks([0,1,2,3,4])
Xstart, Xend = ax.get_xlim()
ax.hlines(y=[0,1,2,3,4], xmin=Xstart, xmax=Xend, color='gray', linewidth = 0.7)
ax.set_yticklabels(CovidStates)

ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.legend(['Mean of last 30 days', 'Number of cases of latest date'], labelcolor=('green', 'coral'), loc = 'upper right')

plt.show()


The above graph shows that the Northern States of Maharashtra and Uttar Pradesh are doing very good in conquering the sudden surge in Covid19 cases.
The Southern States of Karnataka, Kerala and Tamil Nadu are still reeling under the steep rise.

In [ ]:
## Lets bring in the population of each state and find out the ratio of the number of people getting infected and cured.
StatePop = pd.read_csv('../input/population-of-indian-states/StatePopulation.csv')
StatePop.info()

In [ ]:
StatePop.columns = ['State', 'Population']

In [ ]:
## Plotting the statewise % of population affected 

print("Plotting Statewise % of Population Affected\n")

Covid19Pop = pd.merge(Covid19ToDate, StatePop, on = 'State')

Covid19Pop['Cured%'] = (Covid19Pop.Cured/Covid19Pop.Population)* 100
Covid19Pop['Deaths%'] = (Covid19Pop.Deaths/Covid19Pop.Population)* 100
Covid19Pop['Confirmed%'] = (Covid19Pop.Confirmed/Covid19Pop.Population)* 100
Covid19Pop = Covid19Pop.sort_values(by = 'Confirmed', ascending = False)

x = Covid19Pop.State.head(5)
ind = np.arange(5)
w = 0.2

fig, ax = plt.subplots(ncols=1, nrows=1, dpi = 100, figsize=(12,5))
plt.bar(data = Covid19Pop.head(5), x= ind, height= 'Confirmed%', width=w, label = 'Confirmed', color = 'dodgerblue')
plt.bar(data = Covid19Pop.head(5), x= ind+w, height = 'Cured%', width=w, label = 'Cured',color = 'green')
plt.bar(data = Covid19Pop.head(5), x= ind+(2*w), height = 'Deaths%', width=w, label='Deaths',color = 'red')

plt.xticks(ind+w,x)
ax.legend(['Confirmed in %', 'Cured in %', 'Deaths in %'])

for bar in ax.patches:
    ax.annotate(format(bar.get_height(), '.2f'), 
                   (bar.get_x() + bar.get_width() / 2, 
                    bar.get_height()), ha='center', va='center',
                   size=10, xytext=(0, 8),
                   textcoords='offset points')
ax.set_title('% of State Population Affected By Covid-19 Till Date')
ax.set_ylabel('In %')
plt.show()


When taking into consideration of the population of a state, then Kerala seems to be affected more followed by Maharashtra and Karnaka for Confirmed cases.
But the Death % is highest in Maharashtra.

In [ ]:
###################################################################################################
########################### Now we analyse the vaccination details ############################
######################################################################################################

print("Let's analyse the Vaccination Details\n")
CovidVaccine = pd.read_csv('../input/covid19-in-india/covid_vaccine_statewise.csv')
CovidVaccine.info()

In [ ]:
CovidVaccine.rename(columns={'Updated On': 'Date',
                                            'Total Individuals Vaccinated': 'TotalVaccinated',
                                           'First Dose Administered':'FirstDose',
                                           'Second Dose Administered':'SecondDose',
                                           'Male(Individuals Vaccinated)':'Male',
                                           'Female(Individuals Vaccinated)':'Female',
                                           'Transgender(Individuals Vaccinated)':'Transgender',
                                           'Total Covaxin Administered':'TotalCovaxin',
                                           'Total CoviShield Administered':'TotalCoviShield'}, inplace=True)

In [ ]:
print("How many Nan rows of data?\n")
CovidVaccine.isna().sum()

In [ ]:
################################# We drop the rows with Nan values in 'TotalVaccinated' ######################################
CovidVaccine = CovidVaccine[CovidVaccine['TotalVaccinated'].notna()]

In [ ]:
################################## Convert datetime and get the latest date #################################################
CovidVaccine['Date'] = pd.to_datetime(CovidVaccine['Date'], format = '%d/%m/%Y')
latest_date = max(CovidVaccine['Date'])
print("Latest Date : ", latest_date)

In [ ]:
print('There are ',CovidVaccine.State.nunique(), 'unique entries for State column\n')
print('The States are \n\n',CovidVaccine.State.unique())

In [ ]:
CovidVaccine[CovidVaccine['State'] == "Dadra and Nagar Haveli and Daman and Diu"]

In [ ]:
CovidVaccine[CovidVaccine['State'] == "Jammu and Kashmir"]

In [ ]:
CovidVaccine[CovidVaccine['State'] == "Odisha"]

In [ ]:
############################################################################################################################
##################### We change the name of CovidVaccine.State to match with StatePop.State ################################

CovidVaccine.loc[920:1031, 'State'] = 'Dadra and Nagar Haveli & Daman and Diu'
CovidVaccine.loc[1610:1723, 'State'] = 'Jammu and Kashmir + Ladakh'
CovidVaccine.loc[2990:3103, 'State'] = 'Orissa'

#############################################################################################################################
##################### Add the numbers of Jammu and Kashmir with Ladakh of TotalVaccinated, FirstDose, #######################
############################ SecondDose, Male, Female, Transgender, TotalCovaxin, TotalCoviShield ################################

CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['TotalVaccinated'] = CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['TotalVaccinated'] + CovidVaccine[CovidVaccine['State']=='Ladakh']['TotalVaccinated']
CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['FirstDose'] = CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['FirstDose'] + CovidVaccine[CovidVaccine['State']=='Ladakh']['FirstDose']
CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['SecondDose'] = CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['SecondDose'] + CovidVaccine[CovidVaccine['State']=='Ladakh']['SecondDose']
CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['Male'] = CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['Male'] + CovidVaccine[CovidVaccine['State']=='Ladakh']['Male']
CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['Female'] = CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['Female'] + CovidVaccine[CovidVaccine['State']=='Ladakh']['Female']
CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['Transgender'] = CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['Transgender'] + CovidVaccine[CovidVaccine['State']=='Ladakh']['Transgender']
CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['TotalCovaxin'] = CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['TotalCovaxin'] + CovidVaccine[CovidVaccine['State']=='Ladakh']['TotalCovaxin']
CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['TotalCoviShield'] = CovidVaccine[CovidVaccine['State']=='Jammu and Kashmir + Ladakh']['TotalCoviShield'] + CovidVaccine[CovidVaccine['State']=='Ladakh']['TotalCoviShield']
                               
    

In [ ]:
print("After some data cleaning and organising, the rows with Nan values \n")
CovidVaccine.isna().sum()

In [ ]:
########################################## How is India vaccinating ########################################################
print("How is India Vaccinating?\n")
fig, ax = plt.subplots(ncols=1, nrows=1, dpi=100, figsize=(8,6))

sns.lineplot(data=CovidVaccine[CovidVaccine['State']=='India'], x='Date', y='TotalVaccinated', ax=ax)
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(DateFormatter('%b%y'))
ax.set_ylabel('Total Indians Vaccinated so far in millions', fontsize=14)
ax.set_yticklabels([0,0,20,40,60,80,100,120])

Xstart, Xend = ax.get_xlim()
ax.hlines(y=[20e6, 60e6, 100e6], xmin=Xstart, xmax=Xend,  color='gray', linewidth = 0.7)

plt.show()

****From the graph it shows that, till date, around 130 million people have been vaccinated. If we consider the approximate population of India as 1350 million then (130/1350)*100 = 9.62% of the population has been vaccinated till date. Considering that the Vaccination drive for common Indian started from March 01, 2021 and going by the rate of vaccination, it would take 15 months to completele vaccinate India. Or, in other words, India would be able to vaccinate 50% population not before September-October 2021.

In [ ]:
VaccineLatest = CovidVaccine[CovidVaccine['Date'] == latest_date]
StateVaccineLatest = CovidVaccine[(CovidVaccine['State'] != 'India') & (CovidVaccine['Date']==latest_date)]

######################################## Bringing in the State population #############################################


StateVaccineLatest = pd.merge(StateVaccineLatest, StatePop, on ='State')


In [ ]:
########################## ########################## Top 10 states leading in vaccinating its people on percent of population #####################
###################### COMPARED with how much the worst affected States are Vaccinating? ###################################

print(" Top 10 states leading in vaccinating its people on percent of population \n")
print(" COMPARED with how much the worst affected States are Vaccinating? \n")
      
fig, ax = plt.subplots(ncols=2, nrows=1, dpi=100, figsize=(16,8))

StateVaccineLatest['PopVacc']=(StateVaccineLatest.TotalVaccinated/StateVaccineLatest.Population)*100
StateVaccineLatest = StateVaccineLatest.sort_values(by='PopVacc', ascending=False)

sns.barplot(data=StateVaccineLatest.head(10), x='State', y='PopVacc', ax=ax[0])
ax[0].set_ylabel('% of State Population Vaccinated', fontsize=16)
ax[0].set_xlabel('States', fontsize=16)
ax[0].set_xticklabels(StateVaccineLatest.head(10).State, rotation=90, fontsize = 14)
for bar in ax[0].patches:
    ax[0].annotate(format(bar.get_height(), '.1f'), 
                   (bar.get_x() + bar.get_width() / 2, 
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')

data = pd.DataFrame(columns=['State', 'PopVacc'])
for i in range(len(CovidStates)):
    data = data.append(StateVaccineLatest[StateVaccineLatest['State'] == CovidStates[i]][['State', 'PopVacc']])   

sns.barplot(data=data.sort_values(by='PopVacc',ascending=False), x='State', y='PopVacc', ax=ax[1])
ax[1].set_ylabel('% of State Population Vaccinated', fontsize=16)
ax[1].set_xlabel('States', fontsize=20)
ax[1].set_xticklabels(data.sort_values(by='PopVacc',ascending=False).State, rotation=90, fontsize = 14)
for bar in ax[1].patches:
    ax[1].annotate(format(bar.get_height(), '.1f'), 
                   (bar.get_x() + bar.get_width() / 2, 
                    bar.get_height()), ha='center', va='center',
                   size=14, xytext=(0, 8),
                   textcoords='offset points')

Lakshadweep is successful in vaccinating 31.5% of its population, followed by the Eastern Indian States of Sikkim and Tripura.
Of the 5 worst covid affected Indian States, the Southern State of Kerala has vaccinated 18.5% of its population and is among the top 10 Indian States leading in vaccination drive.

In [ ]:
######################## How the Better Vaccinated States are doing in the Corona #############################################

print("How the Better Vaccinated States are doing in Corona?\n")
StateVacc = list(StateVaccineLatest.State.head(5))

fig, ax= plt.subplots(ncols=1, nrows=5, dpi=100, figsize=(14,8))

for i in range(len(StateVacc)):
    StateVaccData = Covid19India[Covid19India['State'] == StateVacc[i]]
    sns.lineplot(data=StateVaccData, x='Date', 
                 y=(StateVaccData.Confirmed-StateVaccData.Confirmed.shift(1)).rolling(window=7).mean(),
                 ax=ax[i], label = StateVacc[i])
    ax[i].set_ylabel(" ")
    ax[i].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax[i].xaxis.set_major_formatter(DateFormatter('%b%y'))

Lakshwadeep and Andaman and Nicobar Islands are showing a dip in Corona cases as compared to the other states.